بسم الله الرحمن الرحيم


Deuxième étape: On a réussi à prendre les livres de fiqh maliki, maintenant on va essayer de mettre l'auteur...

In [239]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'products')
#s = s.find_all('div', class_ = 'products product_list row grid')
#lines = s.find('h3', class_ = 's_title_block flex_child')
#Affichage du contenu de la page
#print(lines)

#Extraire les liens
urls = []
for line in s.findAll('a'):
    urls.append(line.get('href'))#line.text.split(\'n') est une liste de hcaine de caractères, donc [0] est 
    #seulement le str.
    #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n

Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.

In [240]:
#Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
urls = list(np.unique(urls))
#urls

On va supprimer les liens qui ne renvoient pas vers des livres : 
 - Ceux qui finissent pas '.asc' ou 'desc
 - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
 - les "javascript"

In [241]:
#Suppression des liens ne renvoyant pas vers des livres.
urls.remove('javascript:')
urls.remove('javascript:;')
urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')
to_supp = []
for link in range(len(urls)):
    if urls[link].endswith('asc') or urls[link].endswith('.desc') or urls[link].startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?'):
        to_supp.append(urls[link])
    else:
        to_supp.append('')
    

In [242]:
urls = pd.DataFrame(urls)
A = urls == pd.DataFrame(to_supp) 
urls = urls[A == False].dropna()

In [270]:
urls = urls.values.T.tolist()[0]

In [271]:
urls

['https://www.sifatusafwa.com/fr/fiqh-ahkam/charh-bidayah-al-mujtahid-wa-nihayah-al-muqtasid-abdullah-al-abadi.html',
 'https://www.sifatusafwa.com/fr/fiqh-ahkam/tabsirah-al-hukkam-fi-usul-al-aqdhiyyah-ibn-farhun-799h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-jami-li-masa-il-al-mudawwanah-ibn-yunus-as-siqilli-451h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-mou-amalat-fi-al-fiqh-al-maliki-sadiq-al-ghariani.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-muqaddimah-al-qurtubiyyah-bi-charh-cheikh-zarrouk-el-bernoussi.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-usul-fi-ma-rifah-al-usul-risalah-fi-usul-ad-din-ibn-al-arabi.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/charh-aqidah-malik-as-saghir-al-qayrawani-al-qadi-abdelwahhab.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/dalil-as-salik-ala-as-hal-al-massalik-houssain-as-soulaymani.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/hachiyah-ad-dousouqi-ala-ach-charh-al-kabir-fiqh-mali

On va maintenant essayer de regrouper les données des liens dans un dataframe.

## Essai pour le premier lien

In [305]:
url = urls[1]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))

In [306]:
L_string

[['', 'Titre', 'تبصرة الحكام في أصول الأقضية ومناهج الأحكام', ''],
 ['', 'Auteur', 'ابن فرحون المالكي', ''],
 ['', 'Volumes', '3', ''],
 ['', 'Pages', '2013', ''],
 ['', 'Édition', 'دار القلم', ''],
 ['', 'Couverture', 'Rigide', ''],
 ['', 'Vérification et préparation', 'د. عثمان ضميرية', ''],
 ['', 'Volumes', '3', ''],
 ['', 'Format', '17x24cm', ''],
 ['', 'Harakat', 'En Majorité ou en Totalité', '']]

On essaye d'unifier pour enlever les deux "" et afficher en colonnes:

In [307]:
X = []
Y = []
for element in L_string:
    templist = list(np.unique(element))
    templist.remove('')
    X.append(templist[0])
    Y.append(templist[1])

In [308]:
pd.DataFrame({'c': X, "d": Y})

,c,d
0,Titre,تبصرة الحكام في أصول الأقضية ومناهج الأحكام
1,Auteur,ابن فرحون المالكي
2,3,Volumes
3,2013,Pages
4,Édition,دار القلم
5,Couverture,Rigide
6,Vérification et préparation,د. عثمان ضميرية
7,3,Volumes
8,17x24cm,Format
9,En Majorité ou en Totalité,Harakat


On remarque qu'il récupère pas les données dans l'ordre, on aimerait avoir une condition sur des nombres tq:

pas 0, pas 1, 2, 3, pas 4, pas 5, 6, 7, pas 8, pas 9, 10, 11, pas 12, 13 

on remarque que la condition qu'il faut pour les différencier : les nombres qui valent 0 ou 1 modulo 4. (8%4 = 0, 9%4 = 1...) mais ca marche pas pour les autres.

In [309]:
X = []
Y = []
for i in range(len(L_string)):
    templist = list(np.unique(L_string[i]))
    templist.remove('')
    if i%4 == 0 or i%4 ==1:
        X.append(templist[0])
        Y.append(templist[1])
    else:
        X.append(templist[1])
        Y.append(templist[0])


In [310]:
df2 = pd.DataFrame({'c': X, "d": Y})

In [280]:
df.d[0]

'شرح بداية المجتهد ونهاية المقتصد وبهامشه السبيل المرشد'

In [295]:
df = pd.DataFrame({"caracteristiques":X, "test":Y})
df

,caracteristiques,test
0,Titre,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...
1,Auteur,د. عبد الله العبادي
2,Volumes,4
3,Pages,2328
4,Édition,دار السلام
5,Couverture,Rigide
6,Volumes,4
7,Format,17x24cm
8,Harakat,Pas ou peu (ou que sur le Matn)


الحمد لله, il faut maintenant transposer ce tableau et le faire tourner pour tous les sites.

## Tentative de concaténation de deux dataframes

In [296]:
df.set_index("caracteristiques", inplace = True)

In [302]:
df = df.T

In [304]:
df

caracteristiques,Titre,Auteur,Volumes,Pages,Édition,Couverture,Volumes,Format,Harakat
test,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...,د. عبد الله العبادي,4,2328,دار السلام,Rigide,4,17x24cm,Pas ou peu (ou que sur le Matn)


In [319]:
df2 = pd.DataFrame({"caracteristiques":X, "test":Y})
df2.set_index("caracteristiques", inplace = True)
df2 = df2.T

In [330]:
df

caracteristiques,Titre,Auteur,Volumes,Pages,Édition,Couverture,Volumes,Format,Harakat
test,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...,د. عبد الله العبادي,4,2328,دار السلام,Rigide,4,17x24cm,Pas ou peu (ou que sur le Matn)


In [326]:
M = pd.concat((df, df2), axis = 0)

AssertionError: Number of manager items must equal union of block items
# manager items: 11, # tot_items: 12

## Essai de boucle sur tous les sites

In [ ]:
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    for text in s.findAll(class_ ='data-sheet flex_container'):
        L_string.append(text.text.split('\n'))
    L_string
    X = []
    Y = []
    for element in L_string:
        templist = list(np.unique(element))
        templist.remove('')
        X.append(templist[0])
        Y.append(templist[1])
    pd.DataFrame({'c': X, "d": Y})
    X = []
    Y = []
    for i in range(len(L_string)):
        templist = list(np.unique(L_string[i]))
        templist.remove('')
        if i%4 == 0 or i%4 ==1:
            X.append(templist[0])
            Y.append(templist[1])
        else:
            X.append(templist[1])
            Y.append(templist[0])

    df = pd.DataFrame({'c': X, "d": Y})
    df = pd.DataFrame({"caracteristiques":X, "test":Y})
    df
    df.set_index("caracteristiques", inplace = True)
    df.T